In [0]:
%run "./startup"

In [0]:
dbutils.fs.cp('/Volumes/dbx_mit_dev_1udbvf_workspace/default/doimss_carga_archivo/PPA_RCAT069_201906251955040001_3.DAT', 'abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/RCDI/PPA_RCAT069_201906251955040001_3.DAT')

In [0]:
# Crear la instancia con los parámetros esperados
params = WidgetParams({    
    "sr_proceso": str,    
    "sr_subproceso": str,      
    "sr_subetapa": str,
    "sr_id_archivo":str,
    "sr_path_arch" :str
})

# Validar widgets
params.validate()

# 🚀 **Ejemplo de Uso**
conf = ConfManager()
db = DBXConnectionManager()
query = QueryManager()

In [0]:
def load_file(storage_location, sr_path_arch):    
    df = spark.read.text(storage_location + sr_path_arch)
    df = df.withColumnRenamed("value", "FTC_LINEA")
    indexed_df = df.withColumn("idx", monotonically_increasing_id() + 1)
    w = Window.orderBy("idx")
    indexed_df = indexed_df.withColumn("FTN_NO_LINEA", row_number().over(w))
    indexed_df = indexed_df.withColumnRenamed("_c0", "FTC_LINEA")
    indexed_df = indexed_df.drop("idx")
    
    return indexed_df


In [0]:
dfLoaded = load_file(SETTINGS.GENERAL.EXTERNAL_LOCATION,params.sr_path_arch)
if conf.debug:
    display(dfLoaded)

In [0]:
delta_001 = f"delta_001_{params.sr_id_archivo}"
db.write_delta(delta_001, dfLoaded, "overwrite")

In [0]:
statement_demo_001 = query.get_statement(
    "VALIDACION_STRUCT_001.sql",
    SR_ID_ARCHIVO=params.sr_id_archivo,
    SR_SUBETAPA=params.sr_subetapa,
    MAX_LENGTH=conf.val_records_length,
    #FECHA_CARGA=FECHA_CARGA,
    # DELTA_001 = "dbx_mit_dev_1udbvf_workspace.default.delta_001_1"
    DELTA_001=SETTINGS.GENERAL.CATALOG
    + "."
    + SETTINGS.GENERAL.SCHEMA
    + "."
    + delta_001,
)

In [0]:
dfLoaded2 = db.sql_delta(statement_demo_001)
if conf.debug:
    display(dfLoaded2)

In [0]:
delta_002 = f"delta_002_{params.sr_id_archivo}"
db.write_delta(delta_002, dfLoaded2, "overwrite")
if conf.debug:
    display(delta_002)

In [0]:
statement_demo_0003 = query.get_statement(
    "VALIDACION_STRUCT_002.sql",
    temp_view = f"temp_view_001_{params.sr_id_archivo}",
    DELTA_002=SETTINGS.GENERAL.CATALOG
    + "."
    + SETTINGS.GENERAL.SCHEMA
    + "."
    + delta_002,
)

dfLoaded3 = db.sql_delta(statement_demo_0003)

if conf.debug:
    display(dfLoaded3)

In [0]:
db.drop_delta(f"delta_001_{params.sr_id_archivo}")
db.drop_delta(f"delta_002_{params.sr_id_archivo}")